In [1]:
# nb45 | 存檔/讀檔 & 隨機種子重現
# Goals: 遊戲狀態序列化、讀檔復原、隨機種子系統、存檔管理、狀態驗證

# Cell1:  Shared Cache Bootstrap
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

[Cache] ../ai_warehouse/cache | GPU: True


In [ ]:
# Cell 2: 遊戲狀態結構定義
import json
import random
import hashlib
from datetime import datetime
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, asdict
from pathlib import Path


@dataclass
class GameState:
    """Complete game state for save/load"""

    # Core game progress
    chapter: int = 1
    scene: str = "start"
    player_name: str = "Adventurer"

    # Player stats
    hp: int = 100
    max_hp: int = 100
    experience: int = 0
    level: int = 1

    # Inventory & items
    inventory: Dict[str, int] = None  # item_id -> quantity
    equipped: Dict[str, str] = None  # slot -> item_id

    # Skills & abilities
    skills: Dict[str, int] = None  # skill_name -> level
    skill_points: int = 0

    # Story flags & progress
    flags: Dict[str, Any] = None  # story_flags, completed_quests, etc.

    # Meta information
    save_version: str = "1.0"
    random_seed: int = 42
    play_time_minutes: float = 0.0
    save_timestamp: str = ""

    def __post_init__(self):
        # Initialize mutable defaults
        if self.inventory is None:
            self.inventory = {"health_potion": 3, "torch": 1}
        if self.equipped is None:
            self.equipped = {"weapon": "rusty_sword", "armor": "cloth_shirt"}
        if self.skills is None:
            self.skills = {"combat": 1, "exploration": 1, "diplomacy": 1}
        if self.flags is None:
            self.flags = {"tutorial_complete": False, "met_sage": False}

    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for JSON serialization"""
        return asdict(self)

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> "GameState":
        """Create GameState from dictionary"""
        return cls(**data)

    def get_checksum(self) -> str:
        """Generate checksum for save file integrity"""
        state_str = json.dumps(self.to_dict(), sort_keys=True)
        return hashlib.md5(state_str.encode()).hexdigest()

In [ ]:
# Cell 3: 存檔系統實作
class SaveSystem:
    """Handles game save/load operations with versioning"""

    def __init__(self, save_dir: str = "outs/saves"):
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(parents=True, exist_ok=True)

    def save_game(self, state: GameState, slot_name: str = "autosave") -> bool:
        """Save game state to specified slot"""
        try:
            # Update save metadata
            state.save_timestamp = datetime.now().isoformat()

            # Create save data with metadata
            save_data = {
                "game_state": state.to_dict(),
                "checksum": state.get_checksum(),
                "created_at": state.save_timestamp,
                "version": state.save_version,
            }

            # Write to file
            save_path = self.save_dir / f"{slot_name}.json"
            with open(save_path, "w", encoding="utf-8") as f:
                json.dump(save_data, f, indent=2, ensure_ascii=False)

            print(f"✅ Game saved to slot '{slot_name}' at {state.save_timestamp}")
            return True

        except Exception as e:
            print(f"❌ Save failed: {e}")
            return False

    def load_game(self, slot_name: str = "autosave") -> Optional[GameState]:
        """Load game state from specified slot"""
        try:
            save_path = self.save_dir / f"{slot_name}.json"
            if not save_path.exists():
                print(f"❌ Save file '{slot_name}' not found")
                return None

            # Load save data
            with open(save_path, "r", encoding="utf-8") as f:
                save_data = json.load(f)

            # Validate version compatibility
            save_version = save_data.get("version", "1.0")
            if not self._is_compatible_version(save_version):
                print(f"⚠️ Save version {save_version} may be incompatible")

            # Restore game state
            state = GameState.from_dict(save_data["game_state"])

            # Verify checksum integrity
            expected_checksum = save_data.get("checksum", "")
            actual_checksum = state.get_checksum()
            if expected_checksum != actual_checksum:
                print(f"⚠️ Save file checksum mismatch - file may be corrupted")

            print(
                f"✅ Game loaded from slot '{slot_name}' (saved: {save_data.get('created_at', 'unknown')})"
            )
            return state

        except Exception as e:
            print(f"❌ Load failed: {e}")
            return None

    def list_saves(self) -> List[Dict[str, Any]]:
        """List all available save files with metadata"""
        saves = []
        for save_file in self.save_dir.glob("*.json"):
            try:
                with open(save_file, "r", encoding="utf-8") as f:
                    save_data = json.load(f)

                state = save_data["game_state"]
                saves.append(
                    {
                        "slot_name": save_file.stem,
                        "player_name": state.get("player_name", "Unknown"),
                        "level": state.get("level", 1),
                        "chapter": state.get("chapter", 1),
                        "play_time": state.get("play_time_minutes", 0),
                        "saved_at": save_data.get("created_at", "Unknown"),
                        "version": save_data.get("version", "1.0"),
                    }
                )
            except:
                # Skip corrupted files
                continue

        return sorted(saves, key=lambda x: x["saved_at"], reverse=True)

    def delete_save(self, slot_name: str) -> bool:
        """Delete specified save file"""
        try:
            save_path = self.save_dir / f"{slot_name}.json"
            if save_path.exists():
                save_path.unlink()
                print(f"✅ Save '{slot_name}' deleted")
                return True
            else:
                print(f"❌ Save '{slot_name}' not found")
                return False
        except Exception as e:
            print(f"❌ Delete failed: {e}")
            return False

    def _is_compatible_version(self, version: str) -> bool:
        """Check if save version is compatible with current game"""
        current_major = int("1.0".split(".")[0])
        save_major = int(version.split(".")[0])
        return save_major == current_major

In [ ]:
# Cell 4: 隨機種子管理
class SeededRandom:
    """Manages random number generation with reproducible seeds"""

    def __init__(self, seed: int = None):
        self.master_seed = seed or random.randint(1, 1000000)
        self.event_counter = 0
        self.random_state = random.Random(self.master_seed)

    def get_event_seed(self, event_type: str = "general") -> int:
        """Generate deterministic seed for specific event type"""
        self.event_counter += 1
        # Combine master seed, event type hash, and counter for unique but reproducible seed
        event_hash = hash(f"{event_type}_{self.event_counter}")
        return (self.master_seed + event_hash) % 1000000

    def roll_dice(self, sides: int = 20) -> int:
        """Roll dice with current random state"""
        return self.random_state.randint(1, sides)

    def choice(self, items: List[Any]) -> Any:
        """Choose random item with current random state"""
        return self.random_state.choice(items)

    def shuffle(self, items: List[Any]) -> List[Any]:
        """Shuffle list with current random state"""
        shuffled = items.copy()
        self.random_state.shuffle(shuffled)
        return shuffled

    def reset_to_seed(self, seed: int):
        """Reset random state to specific seed"""
        self.master_seed = seed
        self.event_counter = 0
        self.random_state = random.Random(seed)
        print(f"🎲 Random seed reset to {seed}")

In [ ]:
# Cell 5: 遊戲循環整合
class AdventureGame:
    """Main game class with save/load functionality"""

    def __init__(self, save_system: SaveSystem = None):
        self.save_system = save_system or SaveSystem()
        self.state = GameState()
        self.random_gen = SeededRandom(self.state.random_seed)
        self.start_time = datetime.now()

    def new_game(self, player_name: str = "Hero", seed: int = None):
        """Start new game with optional custom seed"""
        if seed is None:
            seed = random.randint(1, 1000000)

        self.state = GameState(player_name=player_name, random_seed=seed)
        self.random_gen = SeededRandom(seed)
        self.start_time = datetime.now()

        print(f"🌟 New game started for {player_name} (seed: {seed})")
        return self.state

    def save_game(self, slot_name: str = "autosave") -> bool:
        """Save current game state"""
        # Update play time
        elapsed = (datetime.now() - self.start_time).total_seconds() / 60
        self.state.play_time_minutes += elapsed
        self.start_time = datetime.now()

        return self.save_system.save_game(self.state, slot_name)

    def load_game(self, slot_name: str = "autosave") -> bool:
        """Load game state from save file"""
        loaded_state = self.save_system.load_game(slot_name)
        if loaded_state:
            self.state = loaded_state
            self.random_gen = SeededRandom(self.state.random_seed)
            self.start_time = datetime.now()
            return True
        return False

    def generate_event(self, event_type: str = "exploration") -> Dict[str, Any]:
        """Generate random event using seeded randomness"""
        event_seed = self.random_gen.get_event_seed(event_type)
        temp_random = random.Random(event_seed)

        # Sample event generation (deterministic based on seed)
        event_types = ["treasure", "combat", "npc_encounter", "puzzle"]
        event = temp_random.choice(event_types)

        events = {
            "treasure": {
                "description": "你發現一個古老的寶箱！",
                "reward": temp_random.choice(["gold", "potion", "artifact"]),
                "amount": temp_random.randint(10, 50),
            },
            "combat": {
                "description": "一隻怪物出現了！",
                "enemy": temp_random.choice(["goblin", "orc", "skeleton"]),
                "difficulty": temp_random.randint(1, 5),
            },
            "npc_encounter": {
                "description": "你遇到了一位旅者",
                "npc_type": temp_random.choice(["merchant", "sage", "wanderer"]),
                "dialogue": temp_random.choice(["friendly", "suspicious", "helpful"]),
            },
            "puzzle": {
                "description": "前方有一個古老的謎題",
                "type": temp_random.choice(["riddle", "mechanism", "symbol"]),
                "difficulty": temp_random.randint(1, 3),
            },
        }

        return {"type": event, "seed": event_seed, **events[event]}

    def take_action(self, action: str) -> Dict[str, Any]:
        """Process player action and update state"""
        # Simple action processing
        result = {"success": True, "message": f"你執行了 {action}"}

        # Update state based on action
        if action == "explore":
            event = self.generate_event("exploration")
            result["event"] = event

        elif action == "rest":
            self.state.hp = min(self.state.hp + 20, self.state.max_hp)
            result["message"] = f"你休息了一下，恢復了體力。目前 HP: {self.state.hp}"

        elif action == "level_up":
            self.state.level += 1
            self.state.skill_points += 3
            result["message"] = f"恭喜！你升到了 {self.state.level} 級"

        # Auto-save every few actions
        if self.random_gen.roll_dice(6) == 1:  # 1/6 chance
            self.save_game("autosave")

        return result

In [ ]:
# Cell 6: **Smoke Test** - 存→讀→驗證狀態一致性
print("=== Smoke Test: Save/Load 功能驗證 ===")

# Initialize game and save system
game = AdventureGame()
save_sys = SaveSystem()

# Create test game state
original_state = game.new_game("TestHero", seed=12345)
print(
    f"原始狀態: Level {original_state.level}, HP {original_state.hp}, Seed {original_state.random_seed}"
)

# Modify game state
game.state.level = 5
game.state.hp = 85
game.state.inventory["magic_scroll"] = 2
game.state.flags["boss_defeated"] = True

print(f"修改後狀態: Level {game.state.level}, HP {game.state.hp}")
print(f"新物品: {game.state.inventory}")

# Save game
save_success = game.save_game("test_save")
print(f"存檔結果: {'成功' if save_success else '失敗'}")

# Generate some events to verify deterministic behavior
print("\n--- 隨機事件生成測試 (seed: 12345) ---")
for i in range(3):
    event = game.generate_event("test")
    print(f"Event {i+1}: {event['type']} (seed: {event['seed']})")

# Create new game instance and load
new_game = AdventureGame()
load_success = new_game.load_game("test_save")
print(f"\n讀檔結果: {'成功' if load_success else '失敗'}")

if load_success:
    loaded_state = new_game.state
    print(f"讀取狀態: Level {loaded_state.level}, HP {loaded_state.hp}")
    print(f"物品清單: {loaded_state.inventory}")
    print(f"故事 Flags: {loaded_state.flags}")

    # Verify random events are deterministic
    print("\n--- 驗證隨機事件重現性 ---")
    for i in range(3):
        event = new_game.generate_event("test")
        print(f"Loaded Event {i+1}: {event['type']} (seed: {event['seed']})")

    # State consistency check
    state_match = (
        original_state.random_seed == loaded_state.random_seed
        and loaded_state.level == 5
        and loaded_state.hp == 85
        and "magic_scroll" in loaded_state.inventory
    )

    print(f"\n✅ 狀態一致性檢查: {'通過' if state_match else '失敗'}")
else:
    print("❌ 無法驗證狀態一致性")

# List all saves
print("\n--- 存檔清單 ---")
all_saves = save_sys.list_saves()
for save in all_saves:
    print(
        f"• {save['slot_name']}: {save['player_name']} Lv.{save['level']} Ch.{save['chapter']} ({save['saved_at'][:19]})"
    )

In [ ]:
# Cell 7: 完整遊戲循環演示
print("\n=== 完整遊戲循環演示 ===")

# Start new adventure
demo_game = AdventureGame()
demo_game.new_game("DemoPlayer", seed=99999)

print("🎮 開始新遊戲...")
print(f"玩家: {demo_game.state.player_name}")
print(f"初始狀態: Lv.{demo_game.state.level}, HP {demo_game.state.hp}/{demo_game.state.max_hp}")

# Simulate gameplay
actions = ["explore", "rest", "explore", "level_up", "explore"]
for i, action in enumerate(actions, 1):
    print(f"\n--- 回合 {i}: {action} ---")
    result = demo_game.take_action(action)
    print(result["message"])

    if "event" in result:
        event = result["event"]
        print(f"隨機事件: {event['description']}")

    # Save at checkpoint
    if i == 3:
        demo_game.save_game("checkpoint")
        print("💾 檢查點存檔完成")

# Demonstrate save/load during gameplay
print(f"\n--- 當前狀態 ---")
print(f"Level: {demo_game.state.level}, HP: {demo_game.state.hp}")
print(f"技能點: {demo_game.state.skill_points}")

# Load from checkpoint
print("\n📁 讀取檢查點...")
demo_game.load_game("checkpoint")
print(f"讀取後狀態: Level {demo_game.state.level}, HP {demo_game.state.hp}")
print(f"技能點: {demo_game.state.skill_points}")

# Continue from checkpoint
print("\n▶️ 從檢查點繼續遊戲...")
final_result = demo_game.take_action("explore")
print(final_result["message"])

# Final save
demo_game.save_game("final_save")
print("💾 最終存檔完成")

# Cleanup test files
save_sys.delete_save("test_save")
save_sys.delete_save("checkpoint")
save_sys.delete_save("final_save")

print("\n🎯 Smoke Test 完成！存檔/讀檔系統運作正常")

In [ ]:
# Cell 8: 總結與最佳實務
print(
    """
=== 總結與最佳實務 ===

✅ 完成功能:
• 完整遊戲狀態序列化 (JSON + checksum)
• 多槽位存檔管理系統
• 隨機種子控制，確保事件可重現
• 版本相容性檢查
• 存檔完整性驗證 (checksum)

🎲 隨機種子策略:
• Master seed + event counter + event type = 可重現但多樣的隨機性
• 每種事件類型有獨立的種子生成
• 支援重置到特定種子進行除錯

💾 存檔管理最佳實務:
• 使用 JSON 格式便於人工檢視和除錯
• 包含 metadata (timestamp, version, checksum)
• 支援多槽位避免玩家意外覆蓋進度
• 自動存檔 + 手動存檔雙重保險

⚠️ 常見陷阱:
• 忘記序列化複雜物件 (需要 to_dict/from_dict)
• 浮點數精度導致的 checksum 不一致
• 版本升級時的向後相容性問題
• 隨機種子在多執行緒環境下的競爭條件

🎯 何時使用:
• 長時間遊戲需要中途保存進度
• 需要支援多玩家或多存檔槽位
• 遊戲包含大量隨機元素但需要可重現性
• 需要分享特定種子的遊戲體驗
• 開發階段需要快速測試特定遊戲狀態
"""
)

In [ ]:
# 存檔目錄位置
SAVE_DIR = "outs/saves"

# 存檔格式版本
SAVE_VERSION = "1.0"

# 自動存檔頻率（動作數）
AUTOSAVE_FREQUENCY = 6  # 每6個動作自動存檔一次

# 檔案完整性檢查
ENABLE_CHECKSUM = True

# 預設種子範圍
SEED_RANGE = (1, 1000000)

# 事件種子生成策略
EVENT_SEED_STRATEGY = "master_seed + event_hash + counter"

# 隨機狀態重置行為
RESET_ON_LOAD = True